<a href="https://colab.research.google.com/github/Hari1128/Hackathon/blob/main/Fluency_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -U openai-whisper librosa soundfile numpy
!pip install phonemizer
!pip install indic-transliteration
!pip install librosa
!pip install numpy

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)


In [31]:
!apt-get install -y espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
espeak is already the newest version (1.48.15+dfsg-3).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [32]:
import re
import whisper
import librosa
import numpy as np
from google.colab import files


In [33]:
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
print("Uploaded file:", audio_path)


Saving briskaudioclip_1.wav to briskaudioclip_1.wav
Uploaded file: briskaudioclip_1.wav


In [34]:
model = whisper.load_model("small")


100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 76.3MiB/s]


In [35]:
result = model.transcribe(
    audio_path,
    word_timestamps=True,
    verbose=False
)

spoken_text = result["text"].lower()
language = result["language"]

print("Detected language:", language)
print("Spoken text:", spoken_text)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: Tamil


100%|██████████| 746/746 [00:23<00:00, 31.64frames/s]

Detected language: ta
Spoken text:  ராஜசேகர் சாவர்த்துக்கும் நாடி தயால்காதில் ஏதோ சொன்னாது யார்காவது தெரியுமா?


In [36]:
def normalize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()

spoken_text = normalize(spoken_text)


In [37]:
words = []

for seg in result["segments"]:
    if "words" in seg:
        for w in seg["words"]:
            if "start" in w and "end" in w:
                words.append({
                    "word": w["word"].strip(),
                    "start": w["start"],
                    "end": w["end"]
                })


In [38]:
pauses = []

for i in range(1, len(words)):
    gap = words[i]["start"] - words[i-1]["end"]
    if gap > 0.25:   # 250 ms pause threshold
        pauses.append(gap)

total_pause_time = sum(pauses)
avg_pause = np.mean(pauses) if pauses else 0


In [39]:
audio_duration = librosa.get_duration(filename=audio_path)
num_words = len(spoken_text.split())

speech_rate = num_words / audio_duration if audio_duration > 0 else 0


/tmp/ipython-input-2467416765.py:1: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  audio_duration = librosa.get_duration(filename=audio_path)


In [40]:
filled_pause_words = {"uh", "um", "er", "ah"}
filled_count = sum(1 for w in spoken_text.split() if w in filled_pause_words)

repetitions = 0
tokens = spoken_text.split()
for i in range(1, len(tokens)):
    if tokens[i] == tokens[i-1]:
        repetitions += 1


In [41]:
def fluency_score(rate, pause_ratio, avg_pause, filled, reps):
    score = 100
    score -= abs(rate - 2.5) * 15
    score -= pause_ratio * 50
    score -= avg_pause * 10
    score -= filled * 5
    score -= reps * 5
    return max(0, round(score, 2))

pause_ratio = total_pause_time / audio_duration if audio_duration > 0 else 0

score = fluency_score(
    speech_rate,
    pause_ratio,
    avg_pause,
    filled_count,
    repetitions
)


In [42]:
def fluency_label(score):
    if score >= 85:
        return "Fluent"
    elif score >= 65:
        return "Mild disfluency"
    elif score >= 40:
        return "Moderate disfluency"
    else:
        return "Severe disfluency"

label = fluency_label(score)


In [43]:
output = {
    "language": language,
    "speech_rate_wps": round(speech_rate, 2),
    "total_pause_time_sec": round(total_pause_time, 2),
    "average_pause_sec": round(avg_pause, 2),
    "pause_ratio": round(pause_ratio, 2),
    "filled_pauses": filled_count,
    "repetitions": repetitions,
    "fluency_score": score,
    "fluency_label": label
}

print("\nFLUENCY ANALYSIS RESULT")
for k, v in output.items():
    print(f"{k}: {v}")



FLUENCY ANALYSIS RESULT
language: ta
speech_rate_wps: 1.07
total_pause_time_sec: 0
average_pause_sec: 0
pause_ratio: 0.0
filled_pauses: 0
repetitions: 0
fluency_score: 78.57
fluency_label: Mild disfluency
